<a href="https://colab.research.google.com/github/FerbeiPatricia/Retele-neuronale2024/blob/main/Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
from torchvision.datasets import MNIST

def relu(z):
    return np.maximum(0, z)

def relu_derivative(z):
    return (z > 0).astype(float)

def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

def cross_entropy_loss(y_pred, y_true):
    m = y_true.shape[0]
    loss = -np.sum(y_true * np.log(y_pred + 1e-9)) / m
    return loss

def forward_propagation(X, W1, b1, W2, b2, dropout_rate=0.2, training=True):

    Z1 = np.dot(X, W1) + b1
    A1 = relu(Z1)

    if training:
        dropout_mask = (np.random.rand(*A1.shape) > dropout_rate).astype(float)
        A1 *= dropout_mask
        A1 /= (1 - dropout_rate)

    Z2 = np.dot(A1, W2) + b2
    y_pred = softmax(Z2)
    return Z1, A1, Z2, y_pred


def backpropagation(X, y_true, Z1, A1, Z2, y_pred, W1, b1, W2, b2, learning_rate):
    m = y_true.shape[0]

    #Z2 = A1 * W2 + b2
    dZ2 = y_pred - y_true
    dW2 = np.dot(A1.T, dZ2) / m
    db2 = np.sum(dZ2, axis=0) / m

    #Z1 = X * W1 + b1
    dA1 = np.dot(dZ2, W2.T)
    dZ1 = dA1 * relu_derivative(Z1)
    dW1 = np.dot(X.T, dZ1) / m
    db1 = np.sum(dZ1, axis=0) / m

    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2

    return W1, b1, W2, b2

def train_neural_network(train_X, train_Y, num_epochs, learning_rate, batch_size=100, dropout_rate=0.5):

    W1 = np.random.randn(train_X.shape[1], 100) * 0.01
    b1 = np.zeros((100,))
    W2 = np.random.randn(100, 10) * 0.01
    b2 = np.zeros((10,))

    num_examples = train_X.shape[0]

    for epoch in range(num_epochs):
        indices = np.arange(num_examples)
        np.random.shuffle(indices)
        train_X = train_X[indices]
        train_Y = train_Y[indices]

        for i in range(0, num_examples, batch_size):
            X_batch = train_X[i:i + batch_size]
            Y_batch = train_Y[i:i + batch_size]

            Z1, A1, Z2, y_pred_batch = forward_propagation(X_batch, W1, b1, W2, b2, dropout_rate=dropout_rate, training=True)

            W1, b1, W2, b2 = backpropagation(X_batch, Y_batch, Z1, A1, Z2, y_pred_batch, W1, b1, W2, b2, learning_rate)

        _, _, _, y_pred_train = forward_propagation(train_X, W1, b1, W2, b2, dropout_rate=dropout_rate, training=False)
        loss = cross_entropy_loss(y_pred_train, train_Y)
        print(f"Epoca {epoch + 1}/{num_epochs}, Loss: {loss:.4f}")

    return W1, b1, W2, b2

def download_mnist(is_train: bool):
    dataset = MNIST(root='./data', transform=lambda x: np.array(x).flatten(), download=True, train=is_train)
    mnist_data = []
    mnist_labels = []
    for image, label in dataset:
        mnist_data.append(image / 255.0)
        mnist_labels.append(label)
    mnist_data = np.array(mnist_data)
    mnist_labels = np.array(mnist_labels)
    mnist_labels_one_hot = np.eye(10)[mnist_labels]
    return mnist_data, mnist_labels_one_hot

train_X, train_Y = download_mnist(True)
test_X, test_Y = download_mnist(False)


num_epochs = 20
learning_rate = 0.1
W1, b1, W2, b2 = train_neural_network(train_X, train_Y, num_epochs, learning_rate, batch_size=100, dropout_rate=0.2)


def test_predict(X, W1, b1, W2, b2):
    _, _, _, y_pred = forward_propagation(X, W1, b1, W2, b2)
    return np.argmax(y_pred, axis=1)

y_test_pred = test_predict(test_X, W1, b1, W2, b2)
test_accuracy = np.mean(np.argmax(test_Y, axis=1) == y_test_pred)
print(f'Acuratețea pe setul de testare : {test_accuracy:.4f}')

Epoca 1/20, Loss: 0.3330
Epoca 2/20, Loss: 0.2674
Epoca 3/20, Loss: 0.2228
Epoca 4/20, Loss: 0.1900
Epoca 5/20, Loss: 0.1690
Epoca 6/20, Loss: 0.1518
Epoca 7/20, Loss: 0.1418
Epoca 8/20, Loss: 0.1297
Epoca 9/20, Loss: 0.1228
Epoca 10/20, Loss: 0.1193
Epoca 11/20, Loss: 0.1179
Epoca 12/20, Loss: 0.1085
Epoca 13/20, Loss: 0.1066
Epoca 14/20, Loss: 0.1007
Epoca 15/20, Loss: 0.1004
Epoca 16/20, Loss: 0.0947
Epoca 17/20, Loss: 0.0932
Epoca 18/20, Loss: 0.0930
Epoca 19/20, Loss: 0.0951
Epoca 20/20, Loss: 0.0895
Acuratețea pe setul de testare : 0.9530
